<a href="https://colab.research.google.com/github/Yangxin666/STD-GAE/blob/scripts/STD_GAE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Spatio-Temporal Denoising Graph Autoencoder (STD-GAE) 

STD-GAE exploits temporal correlation, spatial coherence and value dependencies from domain knowledge to recover missing data. It is empowered by two modules. 

1.   To cope with sparse yet various scenarios of missing data, STD-GAE incorporates a domain-knowledge aware data augmentation module
that creates plausible variations of missing data patterns. This gen-
eralizes STD-GAE to robust imputation over different seasons and
environment.
2.   STD-GAE nontrivially integrates spatiotemporal
graph convolution layers (to recover local missing data by observed
“neighboring” PV plants) and denoising autoencoder (to recover
corrupted data from augmented counterpart) to improve the accu-
racy of imputation accuracy at PV fleet level. 



#Set Up

In [ ]:
# Note that this installation can take a while! Be patient!
!pip install torch-scatter -f https://pytorch-geometric.com/whl/torch-1.10.0+${CUDA}.html
!pip install torch-sparse -f https://pytorch-geometric.com/whl/torch-1.10.0+${CUDA}.html
!pip install torch-cluster -f https://pytorch-geometric.com/whl/torch-1.10.0+${CUDA}.html
!pip install torch-spline-conv -f https://pytorch-geometric.com/whl/torch-1.10.0+${CUDA}.html
!pip install torch-geometric
!pip install torch-geometric-temporal

Looking in links: https://pytorch-geometric.com/whl/torch-1.10.0+.html
  Created wheel for torch-scatter: filename=torch_scatter-2.0.9-cp37-cp37m-linux_x86_64.whl size=279639 sha256=8ea936fd0e071a29f7d2065c65eb9d103213d94ac9455416905a53761ee40ae8
  Stored in directory: /root/.cache/pip/wheels/dd/57/a3/42ea193b77378ce634eb9454c9bc1e3163f3b482a35cdee4d1
Successfully built torch-scatter
Looking in links: https://pytorch-geometric.com/whl/torch-1.10.0+.html
     |████████████████████████████████| 48 kB 4.0 MB/s 
  Created wheel for torch-sparse: filename=torch_sparse-0.6.13-cp37-cp37m-linux_x86_64.whl size=501789 sha256=a761beff830b62f301d1b0f87000aa7fc464ec82f71e050c81edcac995c4ddaa
  Stored in directory: /root/.cache/pip/wheels/e0/01/be/6b2966e0ff20bb023ae35e5d17903e6e5b4df46dd5892f6be6
Successfully built torch-sparse
Looking in links: https://pytorch-geometric.com/whl/torch-1.10.0+.html
     |████████████████████████████████| 43 kB 1.2 MB/s 
  Created wheel for torch-cluster: filename=t

In [ ]:
from datetime import datetime
#import geopy.distance # to compute distances between stations
import glob
import numpy as np
import os
import pandas as pd
import scipy.sparse as sp
from sklearn.preprocessing import StandardScaler
import torch
import torch.nn as nn
import torch.nn.functional as F
#from torch_geometric_temporal.nn import STConv
from tqdm import tqdm
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import ChebConv

from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


#Generate Edge Weight Matrix 

We represent the spatiotemporal PV data as an undirected graph $G = (V, E, X_{t})$. 

1.   Each node in $V$ represents a PV inverter
2.   Edges $E$ are assigned according to Edge Weight Matrix (if $W_{i,j}$ > 0, then there is edge between i and j)
3.   $X_{t}$ denotes a node attribute tensor $\in \mathbb{R}^{T\times n\times d}$. Here T is the length of timeseries, $n$ is the number of nodes (which is 98 in our study), and $d$ is the number of input channel.

Since the locations of PV inverters are fixed, the graph structure is static with time-invariant nodes and edges. 
However, $X_{t}$ is time-varying: each node i carries a  timeseries $x_{i} \in \mathbb{R}^{T\times d}$ recording attributes 
such as temperature, wind speed, 
irradiance and power output.

In [ ]:
from math import radians, cos, sin, asin, sqrt

def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance in kilometers between two points 
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371 # Radius of earth in kilometers. Use 3956 for miles. Determines return value units.
    return c * r

In [ ]:
import pandas as pd
import numpy as np

# please change the path according to your setting
location = pd.read_csv('gdrive/My Drive/STGCN/location.csv',index_col=0)
distance = np.zeros(shape=(98,98))
dist = []
for i in range(98):
    for j in range(98):
        d = haversine(location.iloc[i][2], location.iloc[i][1], location.iloc[j][2], location.iloc[j][1])
        distance[i][j] = d
        dist.append(d)

dist_std = np.std(dist)
distance = pd.DataFrame(distance)
distance

,0,1,2,3,4,5,6,7,8,9,...,88,89,90,91,92,93,94,95,96,97
0,0.000000,41.877922,155.867227,0.000000,0.000000,0.000000,0.000000,19.225334,19.225334,5.266638,...,155.867227,1053.699461,1049.551406,408.413317,408.413317,5.266638,155.867227,41.877922,19.225334,19.225334
1,41.877922,0.000000,197.410750,41.877922,41.877922,41.877922,41.877922,56.183554,56.183554,36.923574,...,197.410750,1042.315749,1038.231327,406.551974,406.551974,36.923574,197.410750,0.000000,56.183554,56.183554
2,155.867227,197.410750,0.000000,155.867227,155.867227,155.867227,155.867227,142.144836,142.144836,161.049981,...,0.000000,1128.415790,1124.103674,470.203780,470.203780,161.049981,0.000000,197.410750,142.144836,142.144836
3,0.000000,41.877922,155.867227,0.000000,0.000000,0.000000,0.000000,19.225334,19.225334,5.266638,...,155.867227,1053.699461,1049.551406,408.413317,408.413317,5.266638,155.867227,41.877922,19.225334,19.225334
4,0.000000,41.877922,155.867227,0.000000,0.000000,0.000000,0.000000,19.225334,19.225334,5.266638,...,155.867227,1053.699461,1049.551406,408.413317,408.413317,5.266638,155.867227,41.877922,19.225334,19.225334
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93,5.266638,36.923574,161.049981,5.266638,5.266638,5.266638,5.266638,23.874407,23.874407,0.000000,...,161.049981,1050.654946,1046.513358,406.295259,406.295259,0.000000,161.049981,36.923574,23.874407,23.874407
94,155.867227,197.410750,0.000000,155.867227,155.867227,155.867227,155.867227,142.144836,142.144836,161.049981,...,0.000000,1128.415790,1124.103674,470.203780,470.203780,161.049981,0.000000,197.410750,142.144836,142.144836
95,41.877922,0.000000,197.410750,41.877922,41.877922,41.877922,41.877922,56.183554,56.183554,36.923574,...,197.410750,1042.315749,1038.231327,406.551974,406.551974,36.923574,197.410750,0.000000,56.183554,56.183554
96,19.225334,56.183554,142.144836,19.225334,19.225334,19.225334,19.225334,0.000000,0.000000,23.874407,...,142.144836,1071.415648,1067.256618,424.409207,424.409207,23.874407,142.144836,56.183554,0.000000,0.000000


In the next step, we compute $w_{ij}$ for each pair of nodes $(i, j)$ in the graph as presented in Yu et al. (2018):

$W_{i,j} = \begin{cases} \exp{(-\frac{d_{ij}^2}{\sigma^2})}, i\neq j \text{ and } \exp{(-\frac{d_{ij}^2}{\sigma^2})} \leq \epsilon \\ 
0 \text{ otherwise }
\end{cases}$

In this case, $d_{ij}$ for each pair of nodes $(i, j)$ is the distance between stations that we have computed above. $\sigma$ is the standard deviation of the distances.

In [ ]:
# epsilon = 0, 0.25, 0.5, 0.75, 1
epsilon = 1
sigma = dist_std
W = np.zeros(shape=(98,98))

for i in range(98):
    for j in range(98):
        if i == j: 
            W[i][j] = 0
        else:
            # Compute distance between stations
            d_ij = distance.loc[i][j]
            
            # Compute weight w_ij
            w_ij = np.exp(-d_ij**2 / sigma**2)
            
            if w_ij >= epsilon:
                W[i, j] = w_ij

W = pd.DataFrame(W)
# please change the path according to your setting
W.to_csv('gdrive/My Drive/STGCN/W_98.csv',index=False)

#Construction of STD-GAE 


*   Construct temporal & spatial layers -> ST Blocks (encoder & decoder) -> Graph Denoising Autoencoders 
*   Define Parameters
*   Some utility functions


Construction of STD-GAE is the foundation for later codes.

Temporal Convolutional (TConv) Layers and Deconvolutional (DeConv) Layers

In [ ]:
class TemporalConv(nn.Module):

    """
    Args:
        in_channels (int): Number of input features.
        out_channels (int): Number of output features.
        kernel_size (int): Convolutional kernel size.
    """

    def __init__(self, in_channels: int, out_channels: int, kernel_size, stride: int, padding: int):
        super(TemporalConv, self).__init__()
        self.conv_1 = nn.Conv2d(in_channels, out_channels, (1, kernel_size), (1, stride), (0,padding))
        self.conv_2 = nn.Conv2d(in_channels, out_channels, (1, kernel_size), (1, stride), (0,padding))
        self.conv_3 = nn.Conv2d(in_channels, out_channels, (1, kernel_size), (1, stride), (0,padding))

    def forward(self, X: torch.FloatTensor) -> torch.FloatTensor:
        """Forward pass through temporal convolution block.

        Arg types:
            * **X** (torch.FloatTensor) -  Input data of shape
                (batch_size, input_time_steps, num_nodes, in_channels).

        Return types:
            * **H** (torch.FloatTensor) - Output data of shape
                (batch_size, in_channels, num_nodes, input_time_steps).
        """
        X = X.permute(0, 3, 2, 1)
        P = self.conv_1(X)
        Q = torch.sigmoid(self.conv_2(X))
        PQ = P * Q
        H = F.relu(PQ + self.conv_3(X))
        H = H.permute(0, 3, 2, 1)
        return H

class TemporalDeConv1(nn.Module):

    """
    Args:
        in_channels (int): Number of input features.
        out_channels (int): Number of output features.
        kernel_size (int): Convolutional kernel size.
    """

    def __init__(self, in_channels: int, out_channels: int, kernel_size, stride: int, padding: int):
        super(TemporalDeConv1, self).__init__()
        self.conv_1 = nn.ConvTranspose2d(in_channels, out_channels, (1, kernel_size), (1, stride), (0,padding))
        self.conv_2 = nn.ConvTranspose2d(in_channels, out_channels, (1, kernel_size),(1, stride), (0,padding))
        self.conv_3 = nn.ConvTranspose2d(in_channels, out_channels, (1, kernel_size),(1, stride), (0,padding))

    def forward(self, X: torch.FloatTensor) -> torch.FloatTensor:
        """Forward pass through temporal convolution block.

        Arg types:
            * **X** (torch.FloatTensor) -  Input data of shape
                (batch_size, input_time_steps, num_nodes, in_channels).

        Return types:
            * **H** (torch.FloatTensor) - Output data of shape
                (batch_size, in_channels, num_nodes, input_time_steps).
        """
        X = X.permute(0, 3, 2, 1)
        P = self.conv_1(X)
        Q = torch.sigmoid(self.conv_2(X))
        PQ = P * Q
        H = F.relu(PQ + self.conv_3(X))
        H = H.permute(0, 3, 2, 1)
        return H

class TemporalDeConv2(nn.Module):

    """
    Args:
        in_channels (int): Number of input features.
        out_channels (int): Number of output features.
        kernel_size (int): Convolutional kernel size.
    """

    def __init__(self, in_channels: int, out_channels: int, kernel_size, stride: int):
        super(TemporalDeConv2, self).__init__()
        self.conv_1 = nn.ConvTranspose2d(in_channels, out_channels, (1, kernel_size),(1, stride))
        self.conv_2 = nn.ConvTranspose2d(in_channels, out_channels, (1, kernel_size),(1, stride))
        self.conv_3 = nn.ConvTranspose2d(in_channels, out_channels, (1, kernel_size),(1, stride))

    def forward(self, X: torch.FloatTensor) -> torch.FloatTensor:
        """Forward pass through temporal convolution block.

        Arg types:
            * **X** (torch.FloatTensor) -  Input data of shape
                (batch_size, input_time_steps, num_nodes, in_channels).

        Return types:
            * **H** (torch.FloatTensor) - Output data of shape
                (batch_size, in_channels, num_nodes, input_time_steps).
        """
        X = X.permute(0, 3, 2, 1)
        P = self.conv_1(X)
        Q = torch.sigmoid(self.conv_2(X))
        PQ = P * Q
        H = F.relu(PQ + self.conv_3(X))
        H = H.permute(0, 3, 2, 1)
        return H


Encoder: a Spaio-temporal Block (Temporal Conv + Spatial Conv + Temporal Conv)

In [ ]:
class STConvEncoder(nn.Module):

    def __init__(
        self,
        num_nodes: int,
        in_channels: int,
        hidden_channels: int,
        out_channels: int,
        kernel_size: int,
        stride: int,
        padding: int,
        K: int,
        normalization: str = "sym",
        bias: bool = True,
    ):
        super(STConvEncoder, self).__init__()
        self.num_nodes = num_nodes
        self.in_channels = in_channels
        self.hidden_channels = hidden_channels
        self.out_channels = out_channels
        self.kernel_size = kernel_size
        self.K = K
        self.normalization = normalization
        self.bias = bias

        self._temporal_conv1 = TemporalConv(
            in_channels=in_channels,
            out_channels=hidden_channels,
            kernel_size=kernel_size, stride = stride, padding = padding,
        )

        self._graph_conv = ChebConv(
            in_channels=hidden_channels,
            out_channels=hidden_channels,
            K=K,
            normalization=normalization,
            bias=bias,
        )

        self._temporal_conv2 = TemporalConv(
            in_channels=hidden_channels,
            out_channels=out_channels,
            kernel_size=kernel_size, stride = stride, padding = padding,
        )

        self._batch_norm = nn.BatchNorm2d(num_nodes)

    def forward(self, X: torch.FloatTensor, edge_index: torch.LongTensor, edge_weight: torch.FloatTensor = None,) -> torch.FloatTensor:

        r"""Forward pass. If edge weights are not present the forward pass
        defaults to an unweighted graph.

        Arg types:
            * **X** (PyTorch FloatTensor) - Sequence of node features of shape (Batch size X Input time steps X Num nodes X In channels).
            * **edge_index** (PyTorch LongTensor) - Graph edge indices.
            * **edge_weight** (PyTorch LongTensor, optional)- Edge weight vector.

        Return types:
            * **T** (PyTorch FloatTensor) - Sequence of node features.
        """
        #print(X.shape)
        T_0 = self._temporal_conv1(X)
        #print(T_0.shape)
        T = torch.zeros_like(T_0).to(T_0.device)
        for b in range(T_0.size(0)):
            for t in range(T_0.size(1)):
                T[b][t] = self._graph_conv(T_0[b][t], edge_index, edge_weight)

        T = F.relu(T)
        #print(T.shape)
        T = self._temporal_conv2(T)
        #print(T.shape)
        # T = T.permute(0, 2, 1, 3)
        # #print(T.shape)
        # T = self._batch_norm(T)
        # T = T.permute(0, 2, 1, 3)
        #print(T.shape)

        return T

Decoder: a Spaio-temporal Block (Temporal DeConv + Spatial Conv + Temporal DeConv)

In [ ]:
class STConvDecoder(nn.Module):

    def __init__(
        self,
        num_nodes: int,
        in_channels: int,
        hidden_channels: int,
        out_channels: int,
        kernel_size: int,
        kernel_size_de: int,
        stride: int,
        padding: int,
        K: int,
        normalization: str = "sym",
        bias: bool = True,
    ):
        super(STConvDecoder, self).__init__()
        self.num_nodes = num_nodes
        self.in_channels = in_channels
        self.hidden_channels = hidden_channels
        self.out_channels = out_channels
        self.kernel_size = kernel_size
        self.K = K
        self.normalization = normalization
        self.bias = bias

        self._temporal_conv1 = TemporalDeConv1(
            in_channels=in_channels,
            out_channels=hidden_channels,
            kernel_size=kernel_size, stride = stride, padding = padding,
        )

        self._graph_conv = ChebConv(
            in_channels=hidden_channels,
            out_channels=hidden_channels,
            K=K,
            normalization=normalization,
            bias=bias,
        )

        self._temporal_conv2 = TemporalDeConv2(
            in_channels=hidden_channels,
            out_channels=out_channels,
            kernel_size=kernel_size_de, stride = stride,
        )

        self._batch_norm = nn.BatchNorm2d(num_nodes)

    def forward(self, X: torch.FloatTensor, edge_index: torch.LongTensor, edge_weight: torch.FloatTensor = None,) -> torch.FloatTensor:

        r"""Forward pass. If edge weights are not present the forward pass
        defaults to an unweighted graph.

        Arg types:
            * **X** (PyTorch FloatTensor) - Sequence of node features of shape (Batch size X Input time steps X Num nodes X In channels).
            * **edge_index** (PyTorch LongTensor) - Graph edge indices.
            * **edge_weight** (PyTorch LongTensor, optional)- Edge weight vector.

        Return types:
            * **T** (PyTorch FloatTensor) - Sequence of node features.
        """
        T_0 = self._temporal_conv1(X)
        T = torch.zeros_like(T_0).to(T_0.device)
        for b in range(T_0.size(0)):
            for t in range(T_0.size(1)):
                T[b][t] = self._graph_conv(T_0[b][t], edge_index, edge_weight)

        T = F.relu(T)
        T = self._temporal_conv2(T)
        # T = T.permute(0, 2, 1, 3)
        # T = self._batch_norm(T)
        # T = T.permute(0, 2, 1, 3)
        return T

Denoising Graph Autoencoder

In [ ]:
# a specified number of STConv blocks, followed by an output layer
class STConvAE(torch.nn.Module):
    def __init__(self, device, num_nodes, channel_size_list, num_layers, 
                 kernel_size, K, window_size, kernel_size_de, stride, padding,\
                 normalization = 'sym', bias = True):
    # num_nodes = number of nodes in the input graph
    # channel_size_list =  2d array representing feature dimensions throughout the model
    # num_layers = number of STConv blocks
    # kernel_size = length of the temporal kernel
    # K = size of the chebyshev filter for the spatial convolution
    # window_size = number of historical time steps to consider

        super(STConvAE, self).__init__()
        self.layers = nn.ModuleList([])
        # add STConv blocks
        for l in range(num_layers):
            input_size, hidden_size, output_size = channel_size_list[l][0], channel_size_list[l][1], channel_size_list[l][2]
            if l==0:
                self.layers.append(STConvEncoder(num_nodes, input_size, hidden_size, output_size, kernel_size, stride, padding, K, normalization, bias))
            if l==1:
                self.layers.append(STConvDecoder(num_nodes, input_size, hidden_size, output_size, kernel_size, kernel_size_de, stride, padding, K, normalization, bias))
        

        # # add output layer
        # self.layers.append(OutputLayer(channel_size_list[-1][-1], \
        #                                window_size - 2 * num_layers * (kernel_size - 1), \
        #                                num_nodes))
        # CUDA if available
        for layer in self.layers:
            layer = layer.to(device)

    def forward(self, x, edge_index, edge_weight ):
        #print(x.shape)
        for layer in self.layers:
            x = layer(x, edge_index, edge_weight)
          #print(x.shape)
        # out_layer = self.layers[-1]
        # x = x.permute(0, 3, 1, 2)
        # x = out_layer(x)
        # print(x.shape)
        return x

Define Parameters

In [ ]:
# model parameters
num_nodes = 98
#channels = np.array([[1, 1, 1], [1, 1, 1]]) # sequence of channel sizes
channels = np.array([[1, 4, 8], [8, 4, 1]])
kernel_size = 4 # size of temporal kernel
kernel_size_de = 2 # size of temporal deconv2
stride = 2
padding = 1
K = 3 # chebyshev filter size

# training parameters
learning_rate = 0.001
batch_size = 2
num_epochs = 50 # note that we trained for 7 epochs using Google Cloud
num_layers = 2 # number of STConv blocks
n_his = 288 # window size
train_prop = 2/3 # Our actual training set proportion was 0.7
val_prop = 1/6 # Our actual training set proportion was 0.2
test_prop = 1/6 # Our actual training set proportion was 0.1

# model save path
model_save_path = os.path.join('best_model_12hr_BM.pt')

Preparing Data

In [ ]:
def data_transform(data, corrupted_data, window, device):
    # data = slice of V matrix
    # n_his = number of historical speed observations to consider
    # n_pred = number of time steps in the future to predict

    num_nodes = data.shape[1]
    num_obs = int(len(data)/window)
    x = np.zeros([num_obs, window, num_nodes, 1])
    y = np.zeros([num_obs, window, num_nodes, 1])
    
    obs_idx = 0
    for i in range(num_obs):
        head = i*window
        tail = (i+1)*window
        y[obs_idx, :, :, :] = data[head: tail].reshape(n_his, num_nodes, 1)
        x[obs_idx, :, :, :] = corrupted_data[head: tail].reshape(n_his, num_nodes, 1)
        #x[obs_idx, :, :, :] = data[head: tail].reshape(n_his, num_nodes, 1)
        obs_idx += 1

    return torch.Tensor(x).to(device), torch.Tensor(y).to(device)

#STD-GAE Framework
STD-GAE framework consists of the following four major components: data ingestion, data augmentation, data corruption, and STD-GAE.

#Data Ingestion

In [ ]:
# please change the path according to your setting
W = pd.read_csv('gdrive/My Drive/STGCN/W_98.csv')
D_O = pd.read_csv('gdrive/My Drive/STGCN/norm_power.csv')
D_O

,0,1,2,3,4,5,6,7,8,9,...,103670,103671,103672,103673,103674,103675,103676,103677,103678,103679
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
94,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
95,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
96,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


#Data Augmentation

In [ ]:
from sklearn.impute import KNNImputer

D_O = D_O.T
imputer = KNNImputer(n_neighbors=5)
D_O = pd.DataFrame(D_O)
imputer.fit(D_O)
D_A = imputer.transform(D_O)
D_A = pd.DataFrame(D_A)
D_A = D_A*100
D_A.to_csv('gdrive/My Drive/STGCN/Data_Augmented.csv', index = False) 
D_A

,0,1,2,3,4,5,6,7,8,9,...,88,89,90,91,92,93,94,95,96,97
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103675,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
103676,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
103677,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
103678,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
D_A.to_csv('gdrive/My Drive/STGCN/Data_Augmented.csv', index = False) 

#Data Corruption

We provide the choice of 12 different missing masks to corrupt, which includes:


1.   Type I: Missing Completely at Random (MCAR): 10%, 20%, 30%, 40%, 50%, and 60%
2.   Type II: Block Missing (BM): 2hrs, 4hrs, 6hrs, 8hrs, 10hrs, and 12hrs.

The missing maks can be adjusted if the prior distribution of missing data is given.



MCAR Masks

Please change the path according to your setting.

In [ ]:
mask = torch.FloatTensor(103680, 98).uniform_() > 0.1
pd.DataFrame(mask.numpy()).to_csv('gdrive/My Drive/STGCN/10%MCAR.csv',index=False)

In [ ]:
mask = torch.FloatTensor(103680, 98).uniform_() > 0.2
pd.DataFrame(mask.numpy()).to_csv('gdrive/My Drive/STGCN/20%MCAR.csv',index=False)

In [ ]:
mask = torch.FloatTensor(103680, 98).uniform_() > 0.3
pd.DataFrame(mask.numpy()).to_csv('gdrive/My Drive/STGCN/30%MCAR.csv',index=False)

In [ ]:
mask = torch.FloatTensor(103680, 98).uniform_() > 0.4
pd.DataFrame(mask.numpy()).to_csv('gdrive/My Drive/STGCN/40%MCAR.csv',index=False)

In [ ]:
mask = torch.FloatTensor(103680, 98).uniform_() > 0.5
pd.DataFrame(mask.numpy()).to_csv('gdrive/My Drive/STGCN/50%MCAR.csv',index=False)

In [ ]:
mask = torch.FloatTensor(103680, 98).uniform_() > 0.6
pd.DataFrame(mask.numpy()).to_csv('gdrive/My Drive/STGCN/60%MCAR.csv',index=False)

BM Masks
Please change the path according to your setting.

In [ ]:
import random

length=24
mask = torch.full((103680,98), True)
for i in range(360):
    for j in range(98):
        number = random.randint(288*i+1,288*(i+1)-length-1)
        mask[number:number+length, j] = False

pd.DataFrame(mask.numpy()).to_csv('gdrive/My Drive/STGCN/2hr_BM.csv',index=False)

In [ ]:
import random

length=48
mask = torch.full((103680,98), True)
for i in range(360):
    for j in range(98):
        number = random.randint(288*i+1,288*(i+1)-length-1)
        mask[number:number+length, j] = False

pd.DataFrame(mask.numpy()).to_csv('gdrive/My Drive/STGCN/4hr_BM.csv',index=False)

In [ ]:
import random

length=72
mask = torch.full((103680,98), True)
for i in range(360):
    for j in range(98):
        number = random.randint(288*i+1,288*(i+1)-length-1)
        mask[number:number+length, j] = False

pd.DataFrame(mask.numpy()).to_csv('gdrive/My Drive/STGCN/6hr_BM.csv',index=False)

In [ ]:
import random

length=96
mask = torch.full((103680,98), True)
for i in range(360):
    for j in range(98):
        number = random.randint(288*i+1,288*(i+1)-length-1)
        mask[number:number+length, j] = False

pd.DataFrame(mask.numpy()).to_csv('gdrive/My Drive/STGCN/8hr_BM.csv',index=False)

In [ ]:
import random

length=120
mask = torch.full((103680,98), True)
for i in range(360):
    for j in range(98):
        number = random.randint(288*i+1,288*(i+1)-length-1)
        mask[number:number+length, j] = False

pd.DataFrame(mask.numpy()).to_csv('gdrive/My Drive/STGCN/10hr_BM.csv',index=False)

In [ ]:
import random

length=144
mask = torch.full((103680,98), True)
for i in range(360):
    for j in range(98):
        number = random.randint(288*i+1,288*(i+1)-length-1)
        mask[number:number+length, j] = False

pd.DataFrame(mask.numpy()).to_csv('gdrive/My Drive/STGCN/12hr_BM.csv',index=False)

In [ ]:
#Choose the mask you want to corrupt D_A: here we choose 12hrs BM
mask = pd.read_csv('gdrive/My Drive/STGCN/12hr_BM.csv')
mask = torch.tensor(mask.values)
D_C = pd.read_csv('gdrive/My Drive/STGCN/Data_Augmented.csv')
D_C[mask.numpy()==False] = -1
D_A = pd.read_csv('gdrive/My Drive/STGCN/Data_Augmented.csv')

#STD-GAE Training

Data Preprocessing

In [ ]:
power_tensor = torch.tensor(D_A.values)
length = D_A.shape[0]
train_x = power_tensor[0:int(train_prop*length),:].to(torch.float32)
validation_x = power_tensor[int(train_prop*length):int((train_prop+val_prop)*length)+1,:].to(torch.float32)
test_x = power_tensor[int((train_prop+val_prop)*length)+1:length,:].to(torch.float32)

power_corrupted_tensor = torch.tensor(D_C.values)
length = D_C.shape[0]
corrupted_train_x = power_corrupted_tensor[0:int(train_prop*length),:].to(torch.float32)
corrupted_validation_x = power_corrupted_tensor[int(train_prop*length):int((train_prop+val_prop)*length)+1,:].to(torch.float32)
corrupted_test_x = power_corrupted_tensor[int((train_prop+val_prop)*length)+1:length,:].to(torch.float32)

In [ ]:
device = torch.device("cuda") if torch.cuda.is_available() \
else torch.device("cpu")

x_train, y_train = data_transform(train_x.numpy(), corrupted_train_x.numpy(), n_his, device)
x_val, y_val = data_transform(validation_x.numpy(), corrupted_validation_x.numpy(), n_his, device)
x_test, y_test = data_transform(test_x.numpy(), corrupted_test_x.numpy(), n_his, device)

# create torch data iterables for training
train_data = torch.utils.data.TensorDataset(x_train, y_train)
train_iter = torch.utils.data.DataLoader(train_data, batch_size, shuffle=True)
val_data = torch.utils.data.TensorDataset(x_val, y_val)
val_iter = torch.utils.data.DataLoader(val_data, batch_size)
test_data = torch.utils.data.TensorDataset(x_test, y_test)
test_iter = torch.utils.data.DataLoader(test_data, batch_size)

# format graph for pyg layer inputs
G = sp.coo_matrix(W)
edge_index = torch.tensor(np.array([G.row, G.col]), dtype=torch.int64).to(device)
edge_weight = torch.tensor(G.data).float().to(device)

Model Training

In [ ]:
model = STConvAE(device, num_nodes, channels, num_layers, kernel_size, K, n_his, kernel_size_de, stride, padding, normalization = 'sym', bias = True).to(device)
# define loss function
loss = nn.MSELoss()
# define optimizer
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate, weight_decay = 0.02) 

In [ ]:
min_valid_loss = np.inf

for epoch in tqdm(range(1, num_epochs + 1), desc = 'Epoch', position = 0):
    train_loss, n = 0.0, 0
    model.train()
    
    for x, y in tqdm(train_iter, desc = 'Batch', position = 0):
        # get model predictions and compute loss
        y_pred = model(x.to(device), edge_index, edge_weight)
        loss = torch.mean((y_pred-y)**2)
        # backpropogation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        train_loss += loss.item()

    valid_loss = 0.0
    model.eval() 
    for x, y in tqdm(val_iter, desc = 'Batch', position = 0):
        # get model predictions and compute loss
        y_pred = model(x.to(device), edge_index, edge_weight)
        loss = torch.mean((y_pred-y)**2)
        valid_loss += loss.item() 

    print(f'Epoch {epoch} \t\t Training Loss: {train_loss/120} \t\t Validation Loss: {valid_loss/30}')
    if min_valid_loss > valid_loss:
        min_valid_loss = valid_loss
        torch.save(model.state_dict(), model_save_path)

Epoch:   2%|▏         | 1/50 [02:11<1:47:19, 131.42s/it]

Epoch 1 		 Training Loss: 464.1682365894318 		 Validation Loss: 806.0076487223307


Epoch:   4%|▍         | 2/50 [04:20<1:44:00, 130.02s/it]

Epoch 2 		 Training Loss: 382.2497326850891 		 Validation Loss: 736.9133895874023


Epoch:   6%|▌         | 3/50 [06:38<1:44:34, 133.50s/it]

...

Epoch 48 		 Training Loss: 48.35977013111115 		 Validation Loss: 91.11478169759114


Epoch:  98%|█████████▊| 49/50 [1:42:06<02:00, 120.72s/it]

Epoch 49 		 Training Loss: 50.96495172182719 		 Validation Loss: 69.28762830098471


Epoch: 100%|██████████| 50/50 [1:44:08<00:00, 124.96s/it]

Epoch 50 		 Training Loss: 49.25429754257202 		 Validation Loss: 64.07692985534668


Evaluate Model

In [ ]:
# load model with lowest validation lost
best_model = STConvAE(device, num_nodes, channels, num_layers, kernel_size, K, n_his, kernel_size_de, stride, padding, normalization = 'sym', bias = True).to(device)
best_model.load_state_dict(torch.load(model_save_path))

best_model.eval()
cost = 0
missing_count = 0
predicted = []
ground_truth = []

i = 1

for x, y in tqdm(test_iter, desc = 'Batch', position = 0):
    # get model predictions and compute loss
    y_pred = best_model(x.to(device), edge_index, edge_weight)
    if i == 1:
        y_pred_complete = y_pred
    else:
        y_pred_complete = torch.cat((y_pred_complete, y_pred), 0)
    i+=1

print(y_pred_complete.shape)

Batch: 100%|██████████| 30/30 [00:18<00:00,  1.60it/s]

torch.Size([60, 288, 98, 1])


In [ ]:
import pandas as pd
dtr = pd.date_range(start='2016-06-27', end='2016-08-26', freq='5min')

In [ ]:
from math import sqrt
pred = y_pred_complete[x_test.cpu().numpy()==-1]
ground_truth = y_test[x_test.cpu().numpy()==-1]
print("Test RMSE of STGCN-DAE is: "+ str(sqrt(torch.mean((pred-ground_truth)**2))))
print("Test MAE is of STGCN-DAE is: "+ str(torch.mean(abs(pred-ground_truth))))
y_pred_complete = torch.squeeze(y_pred_complete)
y_pred_complete = y_pred_complete.reshape(17280,98)
y_pred_complete = pd.DataFrame(y_pred_complete.cpu().detach().numpy())
y_pred_complete['tmst']=dtr[0:17280]
y_pred_complete.to_csv('./Model_results/12hrBM_STGCN_DAE_Imputed.csv', index=False)

Test RMSE of STGCN-DAE is: 11.5665137670197
Test MAE is of STGCN-DAE is: tensor(5.9378, device='cuda:0', grad_fn=<MeanBackward0>)


#Baseline
Only four baselines are here. Scripts for baselines MIDA and LRTC-TNN are ran separately.

LI

In [ ]:
#Linear Interpolation
corrupted_test_x[corrupted_test_x==-1] = np.nan
test = pd.DataFrame(corrupted_test_x.numpy())
LI_imputed = test.interpolate(method ='linear', limit_direction ='forward')
#LI_imputed = LI_imputed.dropna()
LI_imputed = LI_imputed.fillna(LI_imputed.mean())
LI_imputed = torch.tensor(LI_imputed.values)
LI_pred = LI_imputed[~mask[86400:103680,]]
print("Test RMSE of Linear Interpolation is: "+ str(sqrt(torch.mean((LI_pred.cpu()-ground_truth.cpu())**2))))
print("Test MAE of Linear Interpolation is: "+ str(torch.mean(abs(LI_pred.cpu()-ground_truth.cpu()))))

y_pred_complete = pd.DataFrame(LI_imputed.cpu().detach().numpy())
y_pred_complete['tmst']=dtr[0:17280]
y_pred_complete.to_csv('./Model_results/12hrBM_LI_Imputed.csv', index=False)

Test RMSE of Linear Interpolation is: 27.505340612452542
Test MAE of Linear Interpolation is: tensor(20.2995)


Mean

In [ ]:
#Mean Imputation
from math import sqrt
corrupted_test_x[corrupted_test_x==-1] = np.nan
test = pd.DataFrame(corrupted_test_x.numpy())
Mean_imputed = test.fillna(test.mean())
Mean_imputed = torch.tensor(Mean_imputed.values)
Mean_pred = Mean_imputed[~mask[86400:103680,]]
print("Test RMSE of Mean Imputation is: "+ str(sqrt(torch.mean((Mean_pred.cpu()-ground_truth.cpu())**2))))
print("Test MAE of Mean Imputation is: "+ str(torch.mean(abs(Mean_pred.cpu()-ground_truth.cpu()))))

y_pred_complete = pd.DataFrame(Mean_imputed.cpu().detach().numpy())
y_pred_complete['tmst']=dtr[0:17280]
y_pred_complete.to_csv('./Model_results/12hrBM_Mean_Imputed.csv', index=False)

Test RMSE of Mean Imputation is: 37.367874333798575
Test MAE of Mean Imputation is: tensor(28.7702)


MICE

In [ ]:
#MICE
import numpy as np
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor

knn = KNeighborsRegressor()
imp = IterativeImputer(estimator = knn, max_iter = 1, initial_strategy = 'median', imputation_order='ascending',random_state=42)
corrupted_train_x[corrupted_train_x==-1] = np.nan
train = pd.DataFrame(corrupted_train_x.numpy())
imp.fit(train)
corrupted_test_x[corrupted_test_x==-1] = np.nan
test = pd.DataFrame(corrupted_test_x.numpy())
MICE_imputed = imp.transform(test)
MICE_imputed = pd.DataFrame(MICE_imputed)
MICE_imputed = torch.tensor(MICE_imputed.values)
MICE_pred = MICE_imputed[~mask[86400:103680,]]
ground_truth = y_test[x_test.cpu().numpy()==-1]
print("Test RMSE of MICE Interpolation is: "+ str(sqrt(torch.mean((MICE_pred.cpu()-ground_truth.cpu())**2))))
print("Test MAE of MICE Interpolation is: "+ str(torch.mean(abs(MICE_pred.cpu()-ground_truth.cpu()))))

y_pred_complete = pd.DataFrame(MICE_imputed.cpu().detach().numpy())
y_pred_complete['tmst']=dtr[0:17280]
y_pred_complete.to_csv('./Model_results/12hrBM_MICE_Imputed.csv', index=False)

/home/rxf131/ondemand/ubuntu2004/torch-geometric-temporal/sklearn/impute/_iterative.py:699: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


Test RMSE of MICE Interpolation is: 26.79346393194908
Test MAE of MICE Interpolation is: tensor(17.6322)


KNN

In [ ]:
# KNN Imputation
from sklearn.impute import KNNImputer

imputer = KNNImputer(n_neighbors=5)
corrupted_train_x[corrupted_train_x==-1] = np.nan
train = pd.DataFrame(corrupted_train_x.numpy())
imputer.fit(train)
KNN_imputed = imputer.transform(test)
KNN_imputed = pd.DataFrame(KNN_imputed)
KNN_imputed = torch.tensor(KNN_imputed.values)
KNN_pred = KNN_imputed[~mask[86400:103680,]]
print("Test RMSE of KNN Interpolation is: "+ str(sqrt(torch.mean((KNN_pred.cpu()-ground_truth.cpu())**2))))
print("Test MAE of KNN Interpolation is: "+ str(torch.mean(abs(KNN_pred.cpu()-ground_truth.cpu()))))

y_pred_complete = pd.DataFrame(KNN_imputed.cpu().detach().numpy())
y_pred_complete['tmst']=dtr[0:17280]
y_pred_complete.to_csv('./Model_results/12hrBM_KNN_Imputed.csv', index=False)

Test RMSE of KNN Interpolation is: 21.324861672278132
Test MAE of KNN Interpolation is: tensor(13.0600)
